# Quant challenge

Make sure to download the latest __utilities.py__ from Brightspace.

In [ ]:
# Import Python packages
import numpy as np  # Load Numpy package and call it 'np'
import matplotlib.pyplot as plt # Load Pyplot as 'plt'
import utilities as ut # Load the utility functions

## Question 1: Martingale property
Under martingale measure $Q$ every asset $V(t)$ normalized by bank account $B(t)$ is a martingale
$$
\frac{V(0)}{B(0)} = E_0^Q\left[\frac{V(t)}{B(t)}\right]
$$

> - Use Monte Carlo code for the Black-Scholes model below.
> - Under martingale measure $Q$ every asset normalized by bank account is a martingale.
> - Make a __plot__ that would show that stock price $S(t)$ divided by bank account $B(t)$ is a martingale, i.e. show that equation
$$
\frac{S(0)}{B(0)} = E_0^Q\left[\frac{S(t)}{B(t)}\right]
$$
holds for all $t$.

In [ ]:
S0 = 1.     # Stock price
B0 = 1.     # Bank account
r = 0.03    # Risk-free rate 
v = 0.2     # Volatility
T = 1.      # Time to maturity
npath = 10000
nstep = 100
dt = T/nstep
tgrid = np.linspace(0, T, nstep+1)

S = S0*np.ones((nstep+1,npath))
ES = S0/B0 *np.ones(nstep+1)
B = B0*np.exp(r*tgrid)

for i in range(0,nstep):
    ZH = np.random.normal(size=npath//2)
    Z = np.r_[ZH,-ZH] 
    dS = r*S[i,:]*dt + v*np.sqrt(dt)*S[i,:]*Z
    S[i+1,:] = S[i,:] + dS
    ES[i+1] = np.mean(S[i+1,:] / B[i+1]) # Place for your computations

plt.plot(tgrid,ES)
plt.show() 


## Question 2: Martingale property
> - Use Monte Carlo code for the Black-Scholes model below.<br>
> - Under martingale measure $Q$ every asset normalized by bank account is a martingale. <br>
> - Make a __plot__ that would show that European call option price $C(t)$ divided by bank account $B(t)$ is a martingale, i.e. show that equation
$$
\frac{C(0)}{B(0)} = E_0^Q\left[\frac{C(t)}{B(t)}\right]
$$
holds for all $t$.

> - Use function __ut.bsVanilla(K,T,1,S0,r,c,v)__ to compute the call price.

In [ ]:
S0 = 1.     # Stock price
B0 = 1.     # Bank account
K = S0
r = 0.03    # Risk-free rate 
c = 0.0    # Consumtion/dividend rate 
v = 0.2     # Volatility
T = 1.      # Time to maturity
npath = 10000
nstep = 100
dt = T/nstep
tgrid = np.linspace(0, T, nstep+1)
S = S0*np.ones((nstep+1,npath))
ES = S0/B0 * np.ones(nstep+1)
B = B0*np.exp(r*tgrid)
C0 = ut.bsVanilla(K,T,1,S0,r,c,v)
C = C0*np.ones((nstep+1,npath))
EC = C0*np.ones(nstep+1)

for i in range(0,nstep):
    ZH = np.random.normal(size=npath//2)
    Z = np.r_[ZH,-ZH] 
    dS = r*S[i,:]*dt + v*np.sqrt(dt)*S[i,:]*Z
    S[i+1,:] = S[i,:] + dS
    ES[i+1] = np.mean(S[i+1,:] / B[i+1]) # Place for your computations
    tau = T-tgrid[i+1]
    C[i+1,:] = ut.bsVanilla(K,tau,1,S[i+1,:],r,c,v)
    EC[i+1] = np.mean(C[i+1,:] / B[i+1])


plt.plot(tgrid,ES, label = 'E[S]/B')
plt.plot(tgrid,EC / C0, label = 'E[C/C0]/B')
plt.legend()
plt.show() 


## Question 3: Digital (binary) options
Digital call option pays 1 EUR, if stock price is
above strike $K$ at maturity
$$
\Phi(S(T)) = \begin{cases}
1, & \text{if } S(T) \geq K\\
0, & \text{if } S(T) < K
\end{cases}
$$
<!--- ![Digital (binary) option payoff](Digital.png) -->
<img src="Digital.png" alt="Digital (binary) option payoff" width="400"/>

> - For Digital call option
$$
\Phi(S(T)) = \begin{cases}
1, & \text{if } S(T) \geq K\\
0, & \text{if } S(T) < K
\end{cases}
$$
> - Use Black–Scholes model
> - Compute the Monte Carlo price for strike $K = S0 = 1$
> - Compare it to the analytical price using __ut.bsDigital(K,T,1,S0,r,c,v)__

In [ ]:
S0 = 1.     # Stock price
K = S0
r = 0.03    # Risk-free rate 
c = 0.0    # Consumtion/dividend rate 
v = 0.2     # Volatility
T = 1.      # Time to maturity
npath = 10000
nstep = 100
dt = T/nstep
S = S0*np.ones((nstep+1,npath))

for i in range(0,nstep):
    ZH = np.random.normal(size=npath//2)
    Z = np.r_[ZH,-ZH] 
    dS = r*S[i,:]*dt + v*np.sqrt(dt)*S[i,:]*Z
    S[i+1,:] = S[i,:] + dS

ST = S[-1,:]

D_BS_MC = np.exp(-r*T) * np.mean(ST > K)
D_BS_AN = ut.bsDigital(K,T,1,S0,r,c,v)

print(f'MC price: {D_BS_MC}')
print(f'BS price: {D_BS_AN}')


## Question 4: Pricing digital/binary option with smile
> - Use SABR model
> - Compute the Monte Carlo price for strike $K = S0 = 1$
> - Compare it to the analytical price using __sabrVolatility__ to get the volatility at the specific strike and __bsDigital__ to price the option
> - Use __sabrVolatility__ and __bsVanilla__ to compute the right analytical price for the digital option

In [ ]:
S0 = 1.
r = 0.03
c = 0.0
A0 = 0.2
beta = 1
rho = -0.7
eta = 0.5
T = 1.
F0 = S0*np.exp((r-c)*T)
npath = 10000
nstep = 100
dt = T/nstep
CH = np.array([[1., rho], [rho, 1.]])
L = np.linalg.cholesky(CH)
F = F0*np.ones((nstep+1, npath))
A = A0*np.ones((nstep+1, npath))
K = S0
for i in range(0,nstep):
    ZH = np.random.normal(size=(2,npath//2))
    ZA = np.c_[ZH,-ZH] # antithetic sampling
    Z = L @ ZA
    dF = A[i,:] * (F[i,:]**beta) * Z[0,:] * np.sqrt(dt)
    F[i+1,:] = np.maximum(F[i,:] + dF,1e-3)
    dA = A[i,:] * Z[1,:] * eta * np.sqrt(dt)
    A[i+1,:] = np.maximum(A[i,:] + dA,0)
     
ST = F[-1,:]

D_SABR_MC = np.exp(-r*T) * np.mean(ST > K)
vapp = ut.sabrVolatility(S0,K,r,c,A0,beta,rho,eta,T)
D_BS_AN = ut.bsDigital(K,T,1,S0,r,c,vapp)

print(f'MC price: {D_SABR_MC}')
print(f'BS price: {D_BS_AN}')

dK = 0.001*K
K2 = K+dK
vapp2 = ut.sabrVolatility(S0,K2,r,c,A0,beta,rho,eta,T)
D_BS_REP = (ut.bsVanilla(K,T,1,S0,r,c,vapp) - ut.bsVanilla(K2,T,1,S0,r,c,vapp2))/dK
print(f'Replication by the call spread: {D_BS_REP}')